# Gerekli kütüphaneleri yükleyin

In [1]:
import pandas as pd
from lifetimes import BetaGeoFitter, GammaGammaFitter

# Veri setini oku


In [2]:
file_path = '/mnt/data/flo_data_20k.csv'
df = pd.read_csv(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'GitHub/FLO_CLTV_Prediction/flo_data_20k.csv'

# Tarih değişkenlerini date formatına çevir


In [ ]:
date_columns = ["first_order_date", "last_order_date", "last_order_date_online", "last_order_date_offline"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Eksik ve 0 değerleri kontrol et

In [ ]:
print("Eksik değer sayısı:")
print(df.isnull().sum())
print("0 değer sayısı:")
print((df == 0).sum())

# Eksik ve 0 değerleri sil

In [ ]:
df = df.dropna()
df = df[(df[['order_num_total_ever_online', 'order_num_total_ever_offline', 
             'customer_value_total_ever_offline', 'customer_value_total_ever_online']] != 0).all(axis=1)]

# Aykırı Değerleri Baskılamak için Fonksiyonları Tanımla

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

# Belirtilen Değişkenlerin Aykırı Değerlerini Baskıla


In [ ]:
variables = ["order_num_total_ever_online", "order_num_total_ever_offline", 
             "customer_value_total_ever_offline", "customer_value_total_ever_online"]

for variable in variables:
    replace_with_thresholds(df, variable)

# Toplam Alışveriş Sayısı ve Harcaması için Yeni Değişkenler Oluştur


In [ ]:
df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["total_value"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

# Analiz tarihi

In [ ]:
analysis_date = df["last_order_date"].max() + pd.DateOffset(days=2)
print(f'Analiz tarihi: {analysis_date}')

# Recency ve Tenure hesaplama (haftalık olarak)

In [ ]:
df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
df["T_weekly"] = (analysis_date - df["first_order_date"]).dt.days / 7

# Frequency ve Monetary hesaplama

In [ ]:
df["frequency"] = df["total_order"]
df["monetary_cltv_avg"] = df["total_value"] / df["total_order"]

# CLTV dataframe'i oluşturma

In [ ]:
cltv_df = df.loc[:, ["master_id", "recency_cltv_weekly", "T_weekly", "frequency", "monetary_cltv_avg"]]
cltv_df.columns = ["customer_id", "recency_cltv_weekly", "T_weekly", "frequency", "monetary_cltv_avg"]

# Frequency değerlerinin integer olması gerekiyor


In [ ]:
cltv_df["frequency"] = cltv_df["frequency"].astype(int)

# BG/NBD modelini fit et

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])

# 3 ay içerisinde beklenen satın almalar


In [ ]:
cltv_df['exp_sales_3_month'] = bgf.predict(3*4, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])

# 6 ay içerisinde beklenen satın almalar


In [ ]:
cltv_df['exp_sales_6_month'] = bgf.predict(6*4, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])

# Gamma-Gamma modelini fit et


In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

# Tahmin edilen ortalama değeri dataframe'e ekle


In [ ]:
cltv_df['exp_average_value'] = ggf.conditional_expected_average_profit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

# 6 aylık CLTV hesapla


In [ ]:
cltv_df['cltv'] = ggf.customer_lifetime_value(
    bgf, 
    cltv_df['frequency'], 
    cltv_df['recency_cltv_weekly'], 
    cltv_df['T_weekly'], 
    cltv_df['monetary_cltv_avg'], 
    time=6,  # 6 aylık
    freq='W',  # Haftalık
    discount_rate=0.01  # İskonto oranı
)

# Müşterileri CLTV'ye göre segmentlere ayır


In [ ]:
cltv_df['cltv_segment'] = pd.qcut(cltv_df['cltv'], 4, labels=["D", "C", "B", "A"])

# DataFrame'i görüntüle


In [ ]:
print(cltv_df.head())

# CLTV değeri en yüksek 20 kişiyi gözlemle


In [ ]:
top_20_cltv = cltv_df.sort_values(by='cltv', ascending=False).head(20)
print(top_20_cltv)

# Aksiyon önerileri


In [ ]:
recommendations = {
    "Group A": [
        "Sadakat Programları: En değerli müşterilerinize özel sadakat programları sunun. Özel indirimler, erken erişim fırsatları ve ücretsiz kargo gibi avantajlar sağlayarak sadakati artırın.",
        "Kişiselleştirilmiş Pazarlama: Müşterilerin önceki alışveriş alışkanlıklarına dayanarak kişiselleştirilmiş pazarlama kampanyaları oluşturun. Onlara özel ürün önerileri ve kampanyalar sunarak müşteri memnuniyetini artırın."
    ],
    "Group B": [
        "Çapraz Satış Stratejileri: Bu gruptaki müşterilere ilgili ürünleri sunarak çapraz satış fırsatlarını değerlendirin. Örneğin, bir ayakkabı satın alan müşteriye uyumlu çorap veya bakım ürünleri önerin.",
        "Müşteri Geri Bildirimi: Müşteri geri bildirimlerini toplamak ve değerlendirmek için anketler düzenleyin. Müşteri memnuniyetini artırmak için geri bildirimlere dayalı iyileştirmeler yapın."
    ]
}

# Önerileri görüntüle


In [ ]:
print(recommendations)